In [1]:
import urllib
import re
import time
import traceback
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
import time
import os

In [2]:
import pickle

In [3]:
#raw_data = pickle.load(open('./resutls_new.pkl', 'rb'))
raw_data = pickle.load(open('./resutls_20220927.pkl', 'rb'))

In [4]:
import logging
def init_logger(log_file):
    logger = logging.getLogger()
    logger.setLevel('DEBUG')
    basic_format = '%(asctime)s:%(levelname)s:>>>%(message)s'
    date_format = '%Y-%m-%d %H:%M:%S'
    formatter = logging.Formatter(basic_format, date_format)

    chlr = logging.StreamHandler()
    chlr.setLevel('DEBUG')
    chlr.setFormatter(formatter)

    fhlr = logging.FileHandler(log_file)
    fhlr.setLevel('DEBUG')
    fhlr.setFormatter(formatter)

    logger.addHandler(fhlr)
    logger.addHandler(chlr)
    return logger

In [5]:
import re
class Filter:
    def __init__(self, is_filt_dots_prefix=True, is_filt_kuohu_prefix=True, is_filt_dots_end=True):
        self.is_filt_dots_prefix = is_filt_dots_prefix
        self.is_filt_kuohu_prefix = is_filt_kuohu_prefix
        self.is_filt_dots_end = is_filt_dots_end
    def __call__(self, text):
        if self.is_filt_dots_prefix and self.filt_dots_prefix(text):
            return True
        if self.is_filt_kuohu_prefix and self.filt_kuohu_prefix(text):
            return True
        if self.is_filt_dots_end and self.filt_dots_end(text):
            return True
        return False
    def filt_dots_prefix(self, text):
        return text.startswith('.')
    def filt_kuohu_prefix(self, text):
        return text.startswith(')')
    def filt_dots_end(self, text):
        return text.endswith('...')

import re
class Cleaner:
    def __init__(self, is_clean_strip=True, is_clean_utf8=True):
        self.is_clean_strip = is_clean_strip
        self.is_clean_utf8 = is_clean_utf8
    def __call__(self, text):
        if self.is_clean_strip:
            text = self.clean_strip(text)
        if self.is_clean_utf8:
            text = self.clean_utf8(text)
        return text
    def clean_strip(self, text):
        return text.strip()
    def clean_utf8(self,text):
        text = text.replace(u'\uf633', '')
        text = text.replace(u'\u200b', '')
        text = text.replace(u'\u200f', '')
        return text

In [6]:
def spider(query, pn, output_file):
    pn*=10
    keyword = quote(query.encode('gbk'))
    url = f'https://zhidao.baidu.com/search?lm=0&rn=10&pn={pn}&fr=search&ie=gbk&word={keyword}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
        'Cookie': 'BAIDUID=18581A93B7660CCE2EF45F5C9AF78D5D:FG=1; __yjs_duid=1_5a656be8e909a5ab1dc8e6c51a171ff51620382781035; BIDUPSID=18581A93B7660CCE2EF45F5C9AF78D5D; PSTM=1620390429; BDSFRCVID_BFESS=a1POJeC62mInKaverYnGo6QQ_RiR2Y7TH6ao_I6yuPDV6U_ZuVjhEG0PVf8g0KA-hILjogKK3gOTH4PF_2uxOjjg8UtVJeC6EG0Ptf8g0M5; H_BDCLCKID_SF_BFESS=JbPtoD_yfIvafJjYhKTMbJt3-xrHKI62aKDsQI3gBhcqEIL45fJa3-tpbq5xt-juJTQr2prSypT8fxbSj4Qo2x4EQUbyQT3Q5KoaafjSBp5nhMJ_3j7JDMP0qfAjhlcy523iob3vQpPMVhQ3DRoWXPIqbN7P-p5Z5mAqKl0MLPbtbb0xXj_0D6OLDGA8t6ks-DTbB4oHK--_qnTz-4L_5-_e-xQyetJyaR3EWhRvWJ5TMCoz-UoUjqD8K4JmKUuJMeJrBhRgQxKbShPC-tPaj5LFK4oehx7n365EBIO13l02Vbc9e-t2yU_VKHjut4RMW20e0h7mWIbmsxA45J7cM4IseboJLfT-0bc4KKJxbnLWeIJEjjCKj6OyjHLjJTPj-6TKWDP8Kb7VbPTuyUnkbfJBDl5QyMbltKTHKKQXBCb08hR2bPoAeM47yajK25o72TcdK43JbJ7is-8wy5rpQT8rbtFOK5OibCrWBPOTab3vOIOTXpO1jh8zBN5thURB2DkO-4bCWJ5TMl5jDh3Mb6ksD-Ftqj_OtJ-foDKbf-OKJt5kq4bohjPw5mc9BtQmJJu80JRgHRrqqRbT5xFM2RFBDlQNa-0fQg-q3R79fR6-SfoLbJba-JFT-n370x-jLnbOVn0MW-5DVp3gMtnJyUnybPnnBT3T3H8HL4nv2JcJbM5m3x6qLTKkQN3T-PKO5bRu_CF5JIPMbD0menJb5ICq5f7-etJXfKCDbp7F5l8-hR6VhxRH54LXWfTx255ILGONBx_-yC3xOKQphpnzjRF00-OptqQdyjTpBJnN3KJmSRL9bT3vLDuJytvB2-biWb7M2MbdJUJP_IoG2Mn8M4bb3qOpBtQmJeTxoUJ25DnJhhCGe4bK-TryDHtfJfK; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; H_PS_PSSID=34099_31254_33848_33773_33607; delPer=0; BAIDUID_BFESS=EC0B8CB2E5DB2C252FE4F1A4011CDAD8:FG=1; PSINO=2; BA_HECTOR=8ka520a1812g25a0u31gc1pgs0r; ZD_ENTRY=empty; Hm_lvt_6859ce5aaf00fb00387e6434e4fcc925=1621172136,1623254974; Hm_lpvt_6859ce5aaf00fb00387e6434e4fcc925=1623254974; shitong_key_id=2; ab_sr=1.0.1_OTBiZTEzY2RmMWM3MWM1MWNkMDE4MzIwNjllOTJkYmY2M2FiN2UyZGFmYzJjMzllY2FhY2MwMTBjZDI5NzAzZDAxYzVlMzQ2Y2I2YTQyMTkyMWIxNmRhOWUxMTg3ZWU5MGY4Zjc2MmYyOGU0YjIxZmM1ZWMwNzY0ZWYzYzk0MDc1YjFmMTI3ODUyZjg2NmYyNjQ2MDU3MjRjYjg1NjU2Mg==; shitong_data=7d3434b10335de90105aee880b6976138ba670eb5ab5dfec1ec7f6fdead89d6526d65dacb8a6ec84de87e8a3bf8d823c50a8175b16a3745ff826feb07592236f5b646786f633ff511100475de3f136c051560bb904a45f5b5fab2e3376f81eee20967d70e060f03de9793416fa71e0f37a7b315109d2dbfad098fad82b5313db; shitong_sign=86830a0e'
    }

    wb_data = requests.request("GET", url, headers=headers, timeout=10)
    wb_data.encoding = 'gbk'

    html = wb_data.text

    #new_html = html.encode('gbk')

    new_html = html.replace('<br />', '\n').replace('<p>', '\n').replace('</p>', '\n').replace('<br>', '\n')

    #save_path = f'{output_dir}/{query}_{pn}.html'
    save_path = output_file
    logging.info(f'save {query} to {save_path}')
    with open(save_path, 'w') as f:
        ## TODO
        try:
            f.write(new_html)
        except:
            logging.info(f'save {query} to {save_path} error!')
    soup = BeautifulSoup(new_html, 'html.parser')
    ret = []
    for li in soup.find_all('a', attrs={'class':'ti'}):
        ret.append(li.text)
    logging.info(f'get {len(ret)}')
    return ret

def sleep(bound=3, mu=5, sigma=10):
    import time
    import random
    t = -1
    while t<bound:
        t = random.normalvariate(5, 10)
    time.sleep(t)

In [7]:
query_lst = []
def take_all_queries_from_dict(d):
    ret = []
    for k in d:
        dl = d[k]
        if type(dl)==type([]):
            for dq in dl:
                ret.append(dq)
        else:
            ret.extend(take_all_queries_from_dict(dl))
    return ret
for k in raw_data:
    d = raw_data[k]
    ret = take_all_queries_from_dict(d)
    query_lst.extend(ret)
print(len(query_lst))

342077


In [10]:
query_lst = sorted(list(set(query_lst)))

In [11]:
filter_out = Filter()
cleaner = Cleaner()

In [12]:
new_query_lst = [cleaner(q) for q in query_lst if not filter_out(q) ]
query_lst = new_query_lst
query_lst = sorted(list(set(query_lst)))

In [13]:
len(query_lst)

205695

In [14]:
output_dir = './htmls_200w/'
os.system(f'mkdir -p {output_dir}')
query_lst = sorted(query_lst)

In [15]:
logger = init_logger('./log_htmls_200w.txt')

In [33]:
len(results)

2204

In [34]:
#continue_n = 7283
continue_n = 2204

In [35]:
output_dir

'./htmls_200w/'

In [ ]:
for n, query in enumerate(query_lst):
    if n<continue_n:
        continue
    ret = {}
    for pn in [0,1,2,20,70]:
        output_file = f'{output_dir}/{n}_{pn}.html'
        try:
            a = spider(query, pn, output_file)
            ret[pn] = a
        except Exception as e:
            print(e)
        sleep(1,1,10)
    results[query] = ret
    sleep(bound=10)

'gbk' codec can't encode character '\u200e' in position 9: illegal multibyte sequence
'gbk' codec can't encode character '\u200e' in position 9: illegal multibyte sequence
'gbk' codec can't encode character '\u200e' in position 9: illegal multibyte sequence
'gbk' codec can't encode character '\u200e' in position 9: illegal multibyte sequence
'gbk' codec can't encode character '\u200e' in position 9: illegal multibyte sequence


2022-09-30 04:20:59:DEBUG:>>>Starting new HTTPS connection (1): zhidao.baidu.com:443
2022-09-30 04:21:01:DEBUG:>>>https://zhidao.baidu.com:443 "GET /search?lm=0&rn=10&pn=0&fr=search&ie=gbk&word=2022%D6%D0%B9%FA%CA%AE%B4%F3%C3%C5%B4%B0%C6%B7%C5%C6%D3%D0%C4%C4%D0%A9%3F HTTP/1.1" 200 None
2022-09-30 04:21:02:INFO:>>>save 2022中国十大门窗品牌有哪些? to ./htmls_200w//2205_0.html
2022-09-30 04:21:02:INFO:>>>get 11
2022-09-30 04:21:10:DEBUG:>>>Starting new HTTPS connection (1): zhidao.baidu.com:443
2022-09-30 04:21:12:DEBUG:>>>https://zhidao.baidu.com:443 "GET /search?lm=0&rn=10&pn=10&fr=search&ie=gbk&word=2022%D6%D0%B9%FA%CA%AE%B4%F3%C3%C5%B4%B0%C6%B7%C5%C6%D3%D0%C4%C4%D0%A9%3F HTTP/1.1" 200 None
2022-09-30 04:21:12:INFO:>>>save 2022中国十大门窗品牌有哪些? to ./htmls_200w//2205_1.html
2022-09-30 04:21:12:INFO:>>>get 10
2022-09-30 04:21:41:DEBUG:>>>Starting new HTTPS connection (1): zhidao.baidu.com:443
2022-09-30 04:21:43:DEBUG:>>>https://zhidao.baidu.com:443 "GET /search?lm=0&rn=10&pn=20&fr=search&ie=gbk&word=20

2022-09-30 04:26:20:INFO:>>>save 2022事业单位个人工作总结5篇 to ./htmls_200w//2208_70.html
2022-09-30 04:26:20:INFO:>>>get 1
2022-09-30 04:26:56:DEBUG:>>>Starting new HTTPS connection (1): zhidao.baidu.com:443
2022-09-30 04:26:58:DEBUG:>>>https://zhidao.baidu.com:443 "GET /search?lm=0&rn=10&pn=0&fr=search&ie=gbk&word=2022%CA%C2%D2%B5%B5%A5%CE%BB%C8%CB%D4%B1%C4%EA%B6%C8%BF%BC%BA%CB%B9%A4%D7%F7%D7%DC%BD%E1 HTTP/1.1" 200 None
2022-09-30 04:26:58:INFO:>>>save 2022事业单位人员年度考核工作总结 to ./htmls_200w//2209_0.html
2022-09-30 04:26:58:INFO:>>>get 10


In [25]:
n

2204

In [23]:
len(results)
all_p_n = 0
for d in results:
    for k in results[d]:
        all_p_n+=len(results[d][k])

In [126]:
with open('./resutls_20220927.pkl', 'wb') as f:
    pickle.dump(results, f)

In [127]:
all_p_n

342077